In [1]:
# pip install --user gspread oauth2client

In [53]:
import gspread
import time
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [54]:
# folder where the ntuples folder structure is created
ntuples_folder = "/eos/cms/store/group/phys_smp/HadronicVV/signalNTuples_v4/"
backgrounds_folder = "/eos/cms/store/group/phys_smp/HadronicVV/backgroundSamples_v3/"
histo_folder = "/eos/cms/store/group/phys_smp/HadronicVV/signalSamples_v6/"
histo_folder_protonSyst = "/eos/cms/store/group/phys_smp/HadronicVV/signalSamples_v6_protonSystematics/"
histo_folder_JECSystUp = "/eos/cms/store/group/phys_smp/HadronicVV/signalSamples_v6_JECSystematicsUp/"
histo_folder_JECSystDown = "/eos/cms/store/group/phys_smp/HadronicVV/signalSamples_v6_JECSystematicsDown/"


#ntuples_folder = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/signalNTuples_v4/"
#backgrounds_folder = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/backgroundSamples_v2/"
#histo_folder = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/signalSamples_v6/"
#histo_folder_protonSyst = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/signalSamples_v6_protonSystematics/"
#histo_folder_JECSystUp = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/signalSamples_v6_JECSystematicsUp/"
#histo_folder_JECSystDown = "/eos/home-a/abellora/PPtoPPWWjets/MCSamples/tmp/signalSamples_v6_JECSystematicsDown/"

xsec = {
    "WW_A0W5e-7" : 4.2163316591838605E-002,
    "WW_A0W1e-6" : 4.5394743649008480E-002, 
    "WW_A0W2e-6" : 5.8315830794403188E-002, 
    "WW_A0W3_5e-6" : 9.3485285081663066E-002,
    "WW_A0W5e-6" : 0.14813204760228599,
    "WW_ACW2e-6" : 4.2455694489743553E-002,
    "WW_ACW5e-6" : 0.48745879E-01,
    "WW_ACW8e-6" : 6.1402256541154263E-002,
    "WW_ACW1_4e-5" : 0.10203597076732003,
    "WW_ACW2e-5" : 0.16476237329000343,
    "ZZ_A0Z5e-6" : 1.7859991686475674E-002,
    "ZZ_A0Z1e-5" : 7.1475868923898941E-002,
    "ZZ_A0Z2e-5" : 0.28575986698361078,
    "ZZ_A0Z5e-5" : 1.7867409824379850,
    "ZZ_ACZ5e-6" : 1.2752931559334658E-003,
    "ZZ_ACZ1e-5" : 5.1004960918855750E-003,
    "ZZ_ACZ2e-5" : 2.0404690494935453E-002,
    "ZZ_ACZ5e-5" : 0.12751240229708516,
}

lumi = {
    2016 : 4.602909231+1.622461742+3.691150120,
    2017 : 2.361+8.577+4.075+8.948+13.214,
    2018 : 12.103+6.781+6.318+27.669,
}

In [55]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

In [56]:
# get access to the sheet
creds = ServiceAccountCredentials.from_json_keyfile_name("pptoppwwjets-ea629bb33141.json", scope)
client = gspread.authorize(creds)

In [57]:
# open the spreadhseet
sheet = client.open("PPtoPPWWjets_MC")

In [60]:
# get the two worksheets
data_sheet = sheet.worksheet("Data_v6")
results_sheet = sheet.worksheet("Results_v6")
backgrounds_sheet = sheet.worksheet("Backgrounds_v2")

In [61]:
# Fill data spreadsheet for signal

data = data_sheet.get_all_records()
# parse the ntuplizer output file. Example name: 2017/C/ExclWWjets_WW_ACW2e-5_Part1of1.out
# parse the histograms output file. Example name: MC_EventsForMixing_signal_ZZA0Z-1E-5_preTS2_2016B.txt
for i,sample in enumerate(data):
    row = i + 2
    is_produced = True if sample["Status"] == "OK" else False
    is_processed = True if sample["Processed"] == "OK" else False
    year = sample["Year"]
    era  = sample["Era"]
    aqgc = sample["AQGC point"]
    print("Processing",year,era,aqgc)
    sampleNumber = sample["Samplenumber"]
    ntuple_file_name = ntuples_folder + str(year) + "/" + era + "/" + "ExclWWjets_" + aqgc + "_Part1of1.out" 
    histo_file_name = histo_folder + "HadronicWWCuts_" + str(sampleNumber) + ".out"
    histo_file_name_protonSyst = histo_folder_protonSyst + "HadronicWWCuts_" + str(sampleNumber) + ".out"
    histo_file_name_JECSystUp = histo_folder_JECSystUp + "HadronicWWCuts_" + str(sampleNumber) + ".out"
    histo_file_name_JECSystDown = histo_folder_JECSystDown + "HadronicWWCuts_" + str(sampleNumber) + ".out"
    
    if is_produced:
        ntuple_file = open(ntuple_file_name)
        for line in reversed(ntuple_file.readlines()): 
            if line.startswith("TrigReport Events total"):
                nEvents = int(line.split()[4])
                data_sheet.update_cell(row,6,nEvents)
                time.sleep(1)
                break
        ntuple_file.close()
    if is_processed:
        histo_file = open(histo_file_name)
        update = True
        for line in reversed(histo_file.readlines()):
            if line.startswith("Passing ZZ channel cut B ="):
                passingZZ_B = float(line.split()[6])
                # don't update if not necessary
                if (data[i]["Passing ZZ B"] == passingZZ_B) & (data[i]["Passing ZZ B"] != ''):
                    update = False
                    print("Skipping",year,era,aqgc)
                    break
                data_sheet.update_cell(row,32,passingZZ_B)
                time.sleep(3)
            elif line.startswith("Passing ZZ channel cut A ="):
                passingZZ_A = float(line.split()[6])
                data_sheet.update_cell(row,24,passingZZ_A)
                time.sleep(3)
            elif line.startswith("Passing ZZ channel cut ="):
                passingZZ = float(line.split()[5])
                data_sheet.update_cell(row,16,passingZZ)
                time.sleep(3)
            elif line.startswith("Passing WW channel cut B ="):
                passingWW_B = float(line.split()[6])
                data_sheet.update_cell(row,31,passingWW_B)
                time.sleep(3)
            elif line.startswith("Passing WW channel cut A ="):
                passingWW_A = float(line.split()[6])
                data_sheet.update_cell(row,23,passingWW_A)
                time.sleep(3)
            elif line.startswith("Passing WW channel cut ="):
                passingWW = float(line.split()[5])
                data_sheet.update_cell(row,15,passingWW)
                time.sleep(3)
            elif line.startswith("Passing pp-diboson"):
                passingPP_diboson = float(line.split()[4])
                data_sheet.update_cell(row,14,passingPP_diboson)
                time.sleep(3)
            elif line.startswith("Passing 2 PPS"):
                passingPPS = float(line.split()[6])
                data_sheet.update_cell(row,13,passingPPS)
                time.sleep(3)
            elif line.startswith("Passing xi"):
                passingXi = float(line.split()[3])
                data_sheet.update_cell(row,12,passingXi)
                time.sleep(3)
            elif line.startswith("Passing jet"):
                passingJet = float(line.split()[4])
                data_sheet.update_cell(row,11,passingJet)
                time.sleep(3)
            elif line.startswith("Passing dijet"):
                passingDiJet = float(line.split()[4])
                data_sheet.update_cell(row,10,passingDiJet)
                time.sleep(3)
            elif line.startswith("Passing preselection"):
                passingPre = float(line.split()[3])
                data_sheet.update_cell(row,9,passingPre)
                time.sleep(3)
                break
        if update:
            histo_file_protonSyst = open(histo_file_name_protonSyst)
            for line in reversed(histo_file_protonSyst.readlines()):
                if line.startswith("Passing ZZ channel cut B ="):
                    passingZZ_B = float(line.split()[6])
                    data_sheet.update_cell(row,34,passingZZ_B)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut A ="):
                    passingZZ_A = float(line.split()[6])
                    data_sheet.update_cell(row,26,passingZZ_A)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut ="):
                    passingZZ = float(line.split()[5])
                    data_sheet.update_cell(row,18,passingZZ)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut B ="):
                    passingWW_B = float(line.split()[6])
                    data_sheet.update_cell(row,33,passingWW_B)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut A ="):
                    passingWW_A = float(line.split()[6])
                    data_sheet.update_cell(row,25,passingWW_A)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut ="):
                    passingWW = float(line.split()[5])
                    data_sheet.update_cell(row,17,passingWW)
                    time.sleep(3)
                    break
            histo_file_JECSystUp = open(histo_file_name_JECSystUp)
            for line in reversed(histo_file_JECSystUp.readlines()):
                if line.startswith("Passing ZZ channel cut B ="):
                    passingZZ_B = float(line.split()[6])
                    data_sheet.update_cell(row,36,passingZZ_B)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut A ="):
                    passingZZ_A = float(line.split()[6])
                    data_sheet.update_cell(row,28,passingZZ_A)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut ="):
                    passingZZ = float(line.split()[5])
                    data_sheet.update_cell(row,20,passingZZ)
                elif line.startswith("Passing WW channel cut B ="):
                    passingWW_B = float(line.split()[6])
                    data_sheet.update_cell(row,35,passingWW_B)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut A ="):
                    passingWW_A = float(line.split()[6])
                    data_sheet.update_cell(row,27,passingWW_A)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut ="):
                    passingWW = float(line.split()[5])
                    data_sheet.update_cell(row,19,passingWW)
                    break
            histo_file_JECSystDown = open(histo_file_name_JECSystDown)
            for line in reversed(histo_file_JECSystDown.readlines()):
                if line.startswith("Passing ZZ channel cut B ="):
                    passingZZ_B = float(line.split()[6])
                    data_sheet.update_cell(row,38,passingZZ_B)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut A ="):
                    passingZZ_A = float(line.split()[6])
                    data_sheet.update_cell(row,30,passingZZ_A)
                    time.sleep(3)
                elif line.startswith("Passing ZZ channel cut ="):
                    passingZZ = float(line.split()[5])
                    data_sheet.update_cell(row,22,passingZZ)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut B ="):
                    passingWW_B = float(line.split()[6])
                    data_sheet.update_cell(row,37,passingWW_B)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut A ="):
                    passingWW_A = float(line.split()[6])
                    data_sheet.update_cell(row,29,passingWW_A)
                    time.sleep(3)
                elif line.startswith("Passing WW channel cut ="):
                    passingWW = float(line.split()[5])
                    data_sheet.update_cell(row,21,passingWW)
                    time.sleep(3)
                    break

            histo_file.close()
            histo_file_protonSyst.close()
            histo_file_JECSystUp.close()
            histo_file_JECSystDown.close()
            time.sleep(3)

Processing 2016 B WW_A0W5e-7
Skipping 2016 B WW_A0W5e-7
Processing 2016 C WW_A0W5e-7
Skipping 2016 C WW_A0W5e-7
Processing 2016 G WW_A0W5e-7
Skipping 2016 G WW_A0W5e-7
Processing 2017 B WW_A0W5e-7
Skipping 2017 B WW_A0W5e-7
Processing 2017 C WW_A0W5e-7
Skipping 2017 C WW_A0W5e-7
Processing 2017 D WW_A0W5e-7
Skipping 2017 D WW_A0W5e-7
Processing 2017 E WW_A0W5e-7
Skipping 2017 E WW_A0W5e-7
Processing 2017 F WW_A0W5e-7
Skipping 2017 F WW_A0W5e-7
Processing 2018 A WW_A0W5e-7
Skipping 2018 A WW_A0W5e-7
Processing 2018 B WW_A0W5e-7
Skipping 2018 B WW_A0W5e-7
Processing 2018 C WW_A0W5e-7
Skipping 2018 C WW_A0W5e-7
Processing 2018 D WW_A0W5e-7
Skipping 2018 D WW_A0W5e-7
Processing 2016 B WW_A0W1e-6
Skipping 2016 B WW_A0W1e-6
Processing 2016 C WW_A0W1e-6
Skipping 2016 C WW_A0W1e-6
Processing 2016 G WW_A0W1e-6
Skipping 2016 G WW_A0W1e-6
Processing 2017 B WW_A0W1e-6
Skipping 2017 B WW_A0W1e-6
Processing 2017 C WW_A0W1e-6
Skipping 2017 C WW_A0W1e-6
Processing 2017 D WW_A0W1e-6
Skipping 2017 D WW_

Skipping 2016 C ZZ_A0Z1e-5
Processing 2016 G ZZ_A0Z1e-5
Skipping 2016 G ZZ_A0Z1e-5
Processing 2017 B ZZ_A0Z1e-5
Skipping 2017 B ZZ_A0Z1e-5
Processing 2017 C ZZ_A0Z1e-5
Skipping 2017 C ZZ_A0Z1e-5
Processing 2017 D ZZ_A0Z1e-5
Skipping 2017 D ZZ_A0Z1e-5
Processing 2017 E ZZ_A0Z1e-5
Skipping 2017 E ZZ_A0Z1e-5
Processing 2017 F ZZ_A0Z1e-5
Skipping 2017 F ZZ_A0Z1e-5
Processing 2018 A ZZ_A0Z1e-5
Skipping 2018 A ZZ_A0Z1e-5
Processing 2018 B ZZ_A0Z1e-5
Skipping 2018 B ZZ_A0Z1e-5
Processing 2018 C ZZ_A0Z1e-5
Skipping 2018 C ZZ_A0Z1e-5
Processing 2018 D ZZ_A0Z1e-5
Skipping 2018 D ZZ_A0Z1e-5
Processing 2016 B ZZ_A0Z2e-5
Skipping 2016 B ZZ_A0Z2e-5
Processing 2016 C ZZ_A0Z2e-5
Skipping 2016 C ZZ_A0Z2e-5
Processing 2016 G ZZ_A0Z2e-5
Skipping 2016 G ZZ_A0Z2e-5
Processing 2017 B ZZ_A0Z2e-5
Skipping 2017 B ZZ_A0Z2e-5
Processing 2017 C ZZ_A0Z2e-5
Skipping 2017 C ZZ_A0Z2e-5
Processing 2017 D ZZ_A0Z2e-5
Skipping 2017 D ZZ_A0Z2e-5
Processing 2017 E ZZ_A0Z2e-5
Skipping 2017 E ZZ_A0Z2e-5
Processing 2017 F ZZ_

In [63]:
# Fill results spreadsheet for signal

# cross sections from gustavo's page

data = data_sheet.get_all_records()

row_toWrite = 2
firstRun = True
nEvents = 0
aqgc = ""
sum_passingPre = 0
sum_passingDiJet = 0
sum_passingJet = 0
sum_passingXi = 0
sum_passingPPS = 0
sum_passingPP = 0
sum_passingWW = 0
sum_passingWW_protonSyst = 0
sum_passingWW_JECSystUp = 0
sum_passingWW_JECSystDown = 0
sum_passingWW_A = 0
sum_passingWW_protonSyst_A = 0
sum_passingWW_JECSystUp_A = 0
sum_passingWW_JECSystDown_A = 0
sum_passingWW_B = 0
sum_passingWW_protonSyst_B = 0
sum_passingWW_JECSystUp_B = 0
sum_passingWW_JECSystDown_B = 0
sum_passingZZ = 0
sum_passingZZ_protonSyst = 0
sum_passingZZ_JECSystUp = 0
sum_passingZZ_JECSystDown = 0
sum_passingZZ_A = 0
sum_passingZZ_protonSyst_A = 0
sum_passingZZ_JECSystUp_A = 0
sum_passingZZ_JECSystDown_A = 0
sum_passingZZ_B = 0
sum_passingZZ_protonSyst_B = 0
sum_passingZZ_JECSystUp_B = 0
sum_passingZZ_JECSystDown_B = 0
yr_tmp = 0

for yr in [2016,2017,2018]:
    for i,sample in enumerate(data):
        row = i + 2
        is_processed = True if sample["Processed"] == "OK" else False
        year = sample["Year"]
        if year == yr and is_processed:
            if aqgc != sample["AQGC point"] or yr != yr_tmp:
                # write results, unless it's the first time
                if not firstRun:
                    # compute the visible xsec
                    sum_passingPre   = 1000 * xsec[aqgc] * sum_passingPre   / nEvents
                    sum_passingDiJet = 1000 * xsec[aqgc] * sum_passingDiJet / nEvents
                    sum_passingJet   = 1000 * xsec[aqgc] * sum_passingJet   / nEvents
                    sum_passingXi    = 1000 * xsec[aqgc] * sum_passingXi    / nEvents
                    sum_passingPPS   = 1000 * xsec[aqgc] * sum_passingPPS   / nEvents
                    sum_passingPP    = 1000 * xsec[aqgc] * sum_passingPP    / nEvents
                    unc_passingWW    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW) / nEvents
                    sum_passingWW    = 1000 * xsec[aqgc] * sum_passingWW    / nEvents
                    sum_passingWW_protonSyst    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst   / nEvents
                    sum_passingWW_JECSystUp     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp    / nEvents
                    sum_passingWW_JECSystDown   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown  / nEvents
                    unc_passingWW_A    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW_A) / nEvents
                    sum_passingWW_A    = 1000 * xsec[aqgc] * sum_passingWW_A    / nEvents
                    sum_passingWW_protonSyst_A    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst_A   / nEvents
                    sum_passingWW_JECSystUp_A     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp_A    / nEvents
                    sum_passingWW_JECSystDown_A   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown_A  / nEvents
                    unc_passingWW_B    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW_B) / nEvents
                    sum_passingWW_B    = 1000 * xsec[aqgc] * sum_passingWW_B    / nEvents
                    sum_passingWW_protonSyst_B    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst_B   / nEvents
                    sum_passingWW_JECSystUp_B     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp_B    / nEvents
                    sum_passingWW_JECSystDown_B   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown_B  / nEvents
                    unc_passingZZ    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ) / nEvents
                    sum_passingZZ    = 1000 * xsec[aqgc] * sum_passingZZ    / nEvents
                    sum_passingZZ_protonSyst    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst   / nEvents
                    sum_passingZZ_JECSystUp     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp    / nEvents
                    sum_passingZZ_JECSystDown   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown  / nEvents
                    unc_passingZZ_A    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ_A) / nEvents
                    sum_passingZZ_A    = 1000 * xsec[aqgc] * sum_passingZZ_A    / nEvents
                    sum_passingZZ_protonSyst_A    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst_A   / nEvents
                    sum_passingZZ_JECSystUp_A     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp_A    / nEvents
                    sum_passingZZ_JECSystDown_A   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown_A  / nEvents
                    unc_passingZZ_B    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ_B) / nEvents
                    sum_passingZZ_B    = 1000 * xsec[aqgc] * sum_passingZZ_B    / nEvents
                    sum_passingZZ_protonSyst_B    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst_B   / nEvents
                    sum_passingZZ_JECSystUp_B     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp_B    / nEvents
                    sum_passingZZ_JECSystDown_B   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown_B  / nEvents
                    time.sleep(10)
                    results_sheet.update_cell(row_toWrite,3,1000 * xsec[aqgc])
                    results_sheet.update_cell(row_toWrite,4,sum_passingPre)
                    results_sheet.update_cell(row_toWrite,5,sum_passingDiJet)
                    results_sheet.update_cell(row_toWrite,6,sum_passingJet)
                    results_sheet.update_cell(row_toWrite,7,sum_passingXi)
                    results_sheet.update_cell(row_toWrite,8,sum_passingPPS)
                    results_sheet.update_cell(row_toWrite,9,sum_passingPP)
                    results_sheet.update_cell(row_toWrite,10,sum_passingWW)
                    results_sheet.update_cell(row_toWrite,11,sum_passingZZ)
                    time.sleep(10)
                    results_sheet.update_cell(row_toWrite,12,sum_passingWW * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,13,sum_passingZZ * lumi[yr_tmp])
                    if sum_passingWW == 0:
                        results_sheet.update_cell(row_toWrite,14,"inf")
                    else:
                        results_sheet.update_cell(row_toWrite,14,unc_passingWW / sum_passingWW + 1)
                    if sum_passingZZ == 0:
                        results_sheet.update_cell(row_toWrite,15,"inf")
                    else:
                        results_sheet.update_cell(row_toWrite,15,unc_passingZZ / sum_passingZZ + 1)
                    results_sheet.update_cell(row_toWrite,16,sum_passingWW_protonSyst * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,17,sum_passingZZ_protonSyst * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,18,sum_passingWW_JECSystUp * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,19,sum_passingZZ_JECSystUp * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,20,sum_passingWW_JECSystDown * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,21,sum_passingZZ_JECSystDown * lumi[yr_tmp])
                    time.sleep(10)
                    results_sheet.update_cell(row_toWrite,28,sum_passingWW_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,29,sum_passingZZ_A * lumi[yr_tmp])
                    if sum_passingWW_A == 0:
                        results_sheet.update_cell(row_toWrite,30,"inf")
                    else:
                        results_sheet.update_cell(row_toWrite,30,unc_passingWW_A / sum_passingWW_A + 1)
                    if sum_passingZZ_A == 0:
                        results_sheet.update_cell(row_toWrite,31,"inf")
                    else:
                        results_sheet.update_cell(row_toWrite,31,unc_passingZZ_A / sum_passingZZ_A + 1)
                    results_sheet.update_cell(row_toWrite,32,sum_passingWW_protonSyst_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,33,sum_passingZZ_protonSyst_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,34,sum_passingWW_JECSystUp_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,35,sum_passingZZ_JECSystUp_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,36,sum_passingWW_JECSystDown_A * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,37,sum_passingZZ_JECSystDown_A * lumi[yr_tmp])
                    time.sleep(10)
                    results_sheet.update_cell(row_toWrite,44,sum_passingWW_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,45,sum_passingZZ_B * lumi[yr_tmp])
                    if sum_passingWW_B == 0:
                        results_sheet.update_cell(row_toWrite,46,"inf")
                    else:
                        results_sheet.update_cell(row_toWrite,46,unc_passingWW_B / sum_passingWW_B + 1)
                    if sum_passingZZ_B == 0:
                        results_sheet.update_cell(row_toWrite,47,"inf")
                    else:     
                        results_sheet.update_cell(row_toWrite,47,unc_passingZZ_B / sum_passingZZ_B + 1)
                    results_sheet.update_cell(row_toWrite,48,sum_passingWW_protonSyst_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,49,sum_passingZZ_protonSyst_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,50,sum_passingWW_JECSystUp_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,51,sum_passingZZ_JECSystUp_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,52,sum_passingWW_JECSystDown_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,53,sum_passingZZ_JECSystDown_B * lumi[yr_tmp])
                    results_sheet.update_cell(row_toWrite,60,sum_passingWW_A)
                    results_sheet.update_cell(row_toWrite,61,sum_passingZZ_A)
                    results_sheet.update_cell(row_toWrite,62,sum_passingWW_B)
                    results_sheet.update_cell(row_toWrite,63,sum_passingZZ_B)
                    print("Writing "+str(yr_tmp)+" "+aqgc)      
                    yr_tmp = yr
                    nEvents = 0
                    time.sleep(10)
                else:
                    firstRun = False
                    yr_tmp = yr
                    
                # setup for new year and aqgc point
                row_toWrite += 1
                aqgc = sample["AQGC point"]
                nEvents = sample["Events"]
                results_sheet.update_cell(row_toWrite,1,yr_tmp)
                results_sheet.update_cell(row_toWrite,2,aqgc)
                sum_passingPre   = float(sample["Passing preselection"])
                sum_passingDiJet = float(sample["Passing dijet kinematics"])
                sum_passingJet   = float(sample["Passing jet substructure"])
                sum_passingXi    = float(sample["Passing xi(jets)"])
                sum_passingPPS   = float(sample["Passing 2 PPS multiRP"])
                sum_passingPP    = float(sample["Passing pp-diboson matching"])
                sum_passingWW    = float(sample["Passing WW"])
                sum_passingZZ    = float(sample["Passing ZZ"])
                sum_passingWW_protonSyst     = float(sample["Passing WW (protonSyst)"])
                sum_passingZZ_protonSyst     = float(sample["Passing ZZ (protonSyst)"])
                sum_passingWW_JECSystUp      = float(sample["Passing WW (JECSystUp)"])
                sum_passingZZ_JECSystUp      = float(sample["Passing ZZ (JECSystUp)"])
                sum_passingWW_JECSystDown    = float(sample["Passing WW (JECSystDown)"])
                sum_passingZZ_JECSystDown    = float(sample["Passing ZZ (JECSystDown)"])
                sum_passingWW_A    = float(sample["Passing WW A"])
                sum_passingZZ_A    = float(sample["Passing ZZ A"])
                sum_passingWW_protonSyst_A     = float(sample["Passing WW A (protonSyst)"])
                sum_passingZZ_protonSyst_A     = float(sample["Passing ZZ A (protonSyst)"])
                sum_passingWW_JECSystUp_A      = float(sample["Passing WW A (JECSystUp)"])
                sum_passingZZ_JECSystUp_A      = float(sample["Passing ZZ A (JECSystUp)"])
                sum_passingWW_JECSystDown_A    = float(sample["Passing WW A (JECSystDown)"])
                sum_passingZZ_JECSystDown_A    = float(sample["Passing ZZ A (JECSystDown)"])
                sum_passingWW_B    = float(sample["Passing WW B"])
                sum_passingZZ_B    = float(sample["Passing ZZ B"])
                sum_passingWW_protonSyst_B     = float(sample["Passing WW B (protonSyst)"])
                sum_passingZZ_protonSyst_B     = float(sample["Passing ZZ B (protonSyst)"])
                sum_passingWW_JECSystUp_B      = float(sample["Passing WW B (JECSystUp)"])
                sum_passingZZ_JECSystUp_B      = float(sample["Passing ZZ B (JECSystUp)"])
                sum_passingWW_JECSystDown_B    = float(sample["Passing WW B (JECSystDown)"])
                sum_passingZZ_JECSystDown_B    = float(sample["Passing ZZ B (JECSystDown)"])
            else:
                nEvents += sample["Events"]
                sum_passingPre   += float(sample["Passing preselection"])
                sum_passingDiJet += float(sample["Passing dijet kinematics"])
                sum_passingJet   += float(sample["Passing jet substructure"])
                sum_passingXi    += float(sample["Passing xi(jets)"])
                sum_passingPPS   += float(sample["Passing 2 PPS multiRP"])
                sum_passingPP    += float(sample["Passing pp-diboson matching"])
                sum_passingWW    += float(sample["Passing WW"])
                sum_passingZZ    += float(sample["Passing ZZ"])
                sum_passingWW_protonSyst    += float(sample["Passing WW (protonSyst)"])
                sum_passingZZ_protonSyst    += float(sample["Passing ZZ (protonSyst)"]) 
                sum_passingWW_JECSystUp    += float(sample["Passing WW (JECSystUp)"])
                sum_passingZZ_JECSystUp    += float(sample["Passing ZZ (JECSystUp)"]) 
                sum_passingWW_JECSystDown    += float(sample["Passing WW (JECSystDown)"])
                sum_passingZZ_JECSystDown    += float(sample["Passing ZZ (JECSystDown)"]) 
                sum_passingWW_A    += float(sample["Passing WW A"])
                sum_passingZZ_A    += float(sample["Passing ZZ A"])
                sum_passingWW_protonSyst_A    += float(sample["Passing WW A (protonSyst)"])
                sum_passingZZ_protonSyst_A    += float(sample["Passing ZZ A (protonSyst)"]) 
                sum_passingWW_JECSystUp_A    += float(sample["Passing WW A (JECSystUp)"])
                sum_passingZZ_JECSystUp_A    += float(sample["Passing ZZ A (JECSystUp)"]) 
                sum_passingWW_JECSystDown_A    += float(sample["Passing WW A (JECSystDown)"])
                sum_passingZZ_JECSystDown_A    += float(sample["Passing ZZ A (JECSystDown)"])  
                sum_passingWW_B    += float(sample["Passing WW B"])
                sum_passingZZ_B    += float(sample["Passing ZZ B"])
                sum_passingWW_protonSyst_B    += float(sample["Passing WW B (protonSyst)"])
                sum_passingZZ_protonSyst_B    += float(sample["Passing ZZ B (protonSyst)"]) 
                sum_passingWW_JECSystUp_B    += float(sample["Passing WW B (JECSystUp)"])
                sum_passingZZ_JECSystUp_B    += float(sample["Passing ZZ B (JECSystUp)"]) 
                sum_passingWW_JECSystDown_B    += float(sample["Passing WW B (JECSystDown)"])
                sum_passingZZ_JECSystDown_B    += float(sample["Passing ZZ B (JECSystDown)"]) 
                
sum_passingPre   = 1000 * xsec[aqgc] * sum_passingPre   / nEvents
sum_passingDiJet = 1000 * xsec[aqgc] * sum_passingDiJet / nEvents
sum_passingJet   = 1000 * xsec[aqgc] * sum_passingJet   / nEvents
sum_passingXi    = 1000 * xsec[aqgc] * sum_passingXi    / nEvents
sum_passingPPS   = 1000 * xsec[aqgc] * sum_passingPPS   / nEvents
sum_passingPP    = 1000 * xsec[aqgc] * sum_passingPP    / nEvents
unc_passingWW    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW) / nEvents
sum_passingWW    = 1000 * xsec[aqgc] * sum_passingWW    / nEvents
sum_passingWW_protonSyst    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst   / nEvents
sum_passingWW_JECSystUp     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp    / nEvents
sum_passingWW_JECSystDown   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown  / nEvents
unc_passingWW_A    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW_A) / nEvents
sum_passingWW_A    = 1000 * xsec[aqgc] * sum_passingWW_A    / nEvents
sum_passingWW_protonSyst_A    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst_A   / nEvents
sum_passingWW_JECSystUp_A     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp_A    / nEvents
sum_passingWW_JECSystDown_A   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown_A  / nEvents
unc_passingWW_B    = 1000 * xsec[aqgc] * np.sqrt(sum_passingWW_B) / nEvents
sum_passingWW_B    = 1000 * xsec[aqgc] * sum_passingWW_B    / nEvents
sum_passingWW_protonSyst_B    = 1000 * xsec[aqgc] * sum_passingWW_protonSyst_B   / nEvents
sum_passingWW_JECSystUp_B     = 1000 * xsec[aqgc] * sum_passingWW_JECSystUp_B    / nEvents
sum_passingWW_JECSystDown_B   = 1000 * xsec[aqgc] * sum_passingWW_JECSystDown_B  / nEvents
unc_passingZZ    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ) / nEvents
sum_passingZZ    = 1000 * xsec[aqgc] * sum_passingZZ    / nEvents
sum_passingZZ_protonSyst    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst   / nEvents
sum_passingZZ_JECSystUp     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp    / nEvents
sum_passingZZ_JECSystDown   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown  / nEvents
unc_passingZZ_A    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ_A) / nEvents
sum_passingZZ_A    = 1000 * xsec[aqgc] * sum_passingZZ_A    / nEvents
sum_passingZZ_protonSyst_A    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst_A   / nEvents
sum_passingZZ_JECSystUp_A     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp_A    / nEvents
sum_passingZZ_JECSystDown_A   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown_A  / nEvents
unc_passingZZ_B    = 1000 * xsec[aqgc] * np.sqrt(sum_passingZZ_B) / nEvents
sum_passingZZ_B    = 1000 * xsec[aqgc] * sum_passingZZ_B    / nEvents
sum_passingZZ_protonSyst_B    = 1000 * xsec[aqgc] * sum_passingZZ_protonSyst_B   / nEvents
sum_passingZZ_JECSystUp_B     = 1000 * xsec[aqgc] * sum_passingZZ_JECSystUp_B    / nEvents
sum_passingZZ_JECSystDown_B   = 1000 * xsec[aqgc] * sum_passingZZ_JECSystDown_B  / nEvents
results_sheet.update_cell(row_toWrite,3,1000 * xsec[aqgc])
results_sheet.update_cell(row_toWrite,4,sum_passingPre)
results_sheet.update_cell(row_toWrite,5,sum_passingDiJet)
results_sheet.update_cell(row_toWrite,6,sum_passingJet)
results_sheet.update_cell(row_toWrite,7,sum_passingXi)
results_sheet.update_cell(row_toWrite,8,sum_passingPPS)
results_sheet.update_cell(row_toWrite,9,sum_passingPP)
results_sheet.update_cell(row_toWrite,10,sum_passingWW)
results_sheet.update_cell(row_toWrite,11,sum_passingZZ)
time.sleep(10)
results_sheet.update_cell(row_toWrite,12,sum_passingWW * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,13,sum_passingZZ * lumi[yr_tmp])
if sum_passingWW == 0:
    results_sheet.update_cell(row_toWrite,14,"inf")
else:
    results_sheet.update_cell(row_toWrite,14,unc_passingWW / sum_passingWW + 1)
if sum_passingZZ == 0:
    results_sheet.update_cell(row_toWrite,15,"inf")
else:
    results_sheet.update_cell(row_toWrite,15,unc_passingZZ / sum_passingZZ + 1)
results_sheet.update_cell(row_toWrite,16,sum_passingWW_protonSyst * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,17,sum_passingZZ_protonSyst * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,18,sum_passingWW_JECSystUp * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,19,sum_passingZZ_JECSystUp * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,20,sum_passingWW_JECSystDown * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,21,sum_passingZZ_JECSystDown * lumi[yr_tmp])
time.sleep(10)
results_sheet.update_cell(row_toWrite,28,sum_passingWW_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,29,sum_passingZZ_A * lumi[yr_tmp])
if sum_passingWW_A == 0:
    results_sheet.update_cell(row_toWrite,30,"inf")
else:
    results_sheet.update_cell(row_toWrite,30,unc_passingWW_A / sum_passingWW_A + 1)
if sum_passingZZ_A == 0:
    results_sheet.update_cell(row_toWrite,31,"inf")
else:
    results_sheet.update_cell(row_toWrite,31,unc_passingZZ_A / sum_passingZZ_A + 1)
results_sheet.update_cell(row_toWrite,32,sum_passingWW_protonSyst_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,33,sum_passingZZ_protonSyst_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,34,sum_passingWW_JECSystUp_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,35,sum_passingZZ_JECSystUp_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,36,sum_passingWW_JECSystDown_A * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,37,sum_passingZZ_JECSystDown_A * lumi[yr_tmp])
time.sleep(10)
results_sheet.update_cell(row_toWrite,44,sum_passingWW_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,45,sum_passingZZ_B * lumi[yr_tmp])
if sum_passingWW_B == 0:
    results_sheet.update_cell(row_toWrite,46,"inf")
else:
    results_sheet.update_cell(row_toWrite,46,unc_passingWW_B / sum_passingWW_B + 1)
if sum_passingZZ_B == 0:
    results_sheet.update_cell(row_toWrite,47,"inf")
else:     
    results_sheet.update_cell(row_toWrite,47,unc_passingZZ_B / sum_passingZZ_B + 1)
results_sheet.update_cell(row_toWrite,48,sum_passingWW_protonSyst_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,49,sum_passingZZ_protonSyst_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,50,sum_passingWW_JECSystUp_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,51,sum_passingZZ_JECSystUp_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,52,sum_passingWW_JECSystDown_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,53,sum_passingZZ_JECSystDown_B * lumi[yr_tmp])
results_sheet.update_cell(row_toWrite,60,sum_passingWW_A)
results_sheet.update_cell(row_toWrite,61,sum_passingZZ_A)
results_sheet.update_cell(row_toWrite,62,sum_passingWW_B)
results_sheet.update_cell(row_toWrite,63,sum_passingZZ_B)
print("Writing "+str(yr_tmp)+" "+aqgc)

Writing 2016 WW_A0W5e-7
Writing 2016 WW_A0W1e-6
Writing 2016 WW_A0W2e-6
Writing 2016 WW_A0W3_5e-6
Writing 2016 WW_A0W5e-6
Writing 2016 WW_ACW2e-6
Writing 2016 WW_ACW2e-5
Writing 2016 WW_ACW5e-6
Writing 2016 WW_ACW8e-6
Writing 2016 WW_ACW1_4e-5
Writing 2016 WW_ACW2e-5
Writing 2016 ZZ_A0Z5e-6
Writing 2016 ZZ_A0Z1e-5
Writing 2016 ZZ_A0Z2e-5
Writing 2016 ZZ_A0Z5e-5
Writing 2016 ZZ_ACZ5e-6
Writing 2016 ZZ_ACZ1e-5
Writing 2016 ZZ_ACZ2e-5
Writing 2016 ZZ_ACZ5e-5
Writing 2017 WW_A0W5e-7
Writing 2017 WW_A0W1e-6
Writing 2017 WW_A0W2e-6
Writing 2017 WW_A0W3_5e-6
Writing 2017 WW_A0W5e-6
Writing 2017 WW_ACW2e-6
Writing 2017 WW_ACW2e-5
Writing 2017 WW_ACW5e-6
Writing 2017 WW_ACW8e-6
Writing 2017 WW_ACW1_4e-5
Writing 2017 WW_ACW2e-5
Writing 2017 ZZ_A0Z5e-6
Writing 2017 ZZ_A0Z1e-5
Writing 2017 ZZ_A0Z2e-5
Writing 2017 ZZ_A0Z5e-5
Writing 2017 ZZ_ACZ5e-6
Writing 2017 ZZ_ACZ1e-5
Writing 2017 ZZ_ACZ2e-5
Writing 2017 ZZ_ACZ5e-5
Writing 2018 WW_A0W5e-7
Writing 2018 WW_A0W1e-6
Writing 2018 WW_A0W2e-6
Writing 

In [46]:
xsecAndSamplenumber = {
    2:6838*1.14405,
    3:551.1*1.17619,
    4:156.4*1.19501,
    5:32.293,
    6:7.466*1.26149,
    7:377.96,
    8:33.7,
    9:14.6,
    10:0.6481*1.30019,
    102:6830.0,
    103:642.1,
    104:185.9,
    105:32.293,
    106:9.4183,
    107:21.3,
    108:41.3,
    109:21.3,
    110:0.84265,
    111:66.85,
    202:6830.0,
    203:642.1,
    204:185.9,
    205:32.293,
    206:9.4183,
    207:377.96,
    208:33.7,
    209:14.6,
    210:0.84265,    
} # pb
bg_xsec_tot = {
    "2016":0,
    "2017":0,
    "2018":0,
}
for key,value in xsecAndSamplenumber.items():
    if key < 100:
        bg_xsec_tot["2017"] += value
    elif key < 200:
        bg_xsec_tot["2016"] += value
    elif key < 300:
        bg_xsec_tot["2018"] += value       
print(bg_xsec_tot)

{'2016': 7851.30395, '2017': 9126.925710478998, '2018': 8126.81395}


In [47]:
# Fill backgrounds spreadsheet

data = backgrounds_sheet.get_all_records()
# parse the ntuplizer output file. Example name: 2017/C/ExclWWjets_WW_ACW2e-5_Part1of1.out
# parse the histograms output file. Example name: MC_EventsForMixing_signal_ZZA0Z-1E-5_preTS2_2016B.txt

for i,sample in enumerate(data):
    row = i + 2
    is_processed = True if sample["Processed"] == "OK" else False
    if sample["Era"] == "":
        pass
    year = int(sample["Year"])
    era  = sample["Era"]
    aqgc = sample["AQGC point"]
    events = sample["Events"]
    print("Processing",year,aqgc)
    sampleNumber = int(sample["Samplenumber"])
    histo_file_name = backgrounds_folder + "HadronicWWCuts_" + str(sampleNumber) + ".out"
    
    if is_processed:
        histo_file = open(histo_file_name)
        update = True
        for line in reversed(histo_file.readlines()):
            if line.startswith("Passing WW channel before PPS matching"):
                passingWWbeforePPS = float(line.split()[7])
                backgrounds_sheet.update_cell(row,39,passingWWbeforePPS*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)
            elif line.startswith("Passing ZZ channel before PPS matching"):
                passingZZbeforePPS = float(line.split()[7])
                backgrounds_sheet.update_cell(row,40,passingZZbeforePPS*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)                
            elif line.startswith("Passing xi"):
                passingXi = float(line.split()[3])
                backgrounds_sheet.update_cell(row,12,passingXi*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)
            elif line.startswith("Passing jet"):
                passingJet = float(line.split()[4])
                backgrounds_sheet.update_cell(row,11,passingJet*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)
            elif line.startswith("Passing dijet"):
                passingDiJet = float(line.split()[4])
                backgrounds_sheet.update_cell(row,10,passingDiJet*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)
            elif line.startswith("Passing preselection"):
                passingPre = float(line.split()[3])
                backgrounds_sheet.update_cell(row,9,passingPre*xsecAndSamplenumber[sampleNumber]*1000*lumi[year]/events)
                time.sleep(1)
                break

            histo_file.close()

Processing 2017 qcdpt300to470
Processing 2017 qcdpt470to600
Processing 2017 qcdpt600to800
Processing 2017 qcdpt800to1000
Processing 2017 qcdpt1000to1400
Processing 2017 ttbarhadronic
Processing 2017 wjetshadronic
Processing 2017 zjetshadronic
Processing 2017 qcdpt1400to1800
Processing 2016 qcdpt300to470
Processing 2016 qcdpt470to600
Processing 2016 qcdpt600to800
Processing 2016 qcdpt800to1000
Processing 2016 qcdpt1000to1400
Processing 2016 ttbarhadronic1000inf
Processing 2016 wjetshadronic
Processing 2016 zjetshadronic
Processing 2016 qcdpt1400to1800
Processing 2016 ttbarhadronic700ot1000
Processing 2018 qcdpt300to470
Processing 2018 qcdpt470to600
Processing 2018 qcdpt600to800
Processing 2018 qcdpt800to1000
Processing 2018 qcdpt1000to1400
Processing 2018 ttbarhadronic
Processing 2018 wjetshadronic
Processing 2018 zjetshadronic
Processing 2018 qcdpt1400to1800


In [25]:
# Produce results tables
results_sheet = sheet.worksheet("Results_v6")
results = results_sheet.get_all_records(head=2)
bg_sheet = sheet.worksheet("Backgrounds_v2")
bgs = bg_sheet.get_all_records(head=1)


In [26]:
name_and_cell_WW = {
    "Generator-level (all decay channels)" : "Generator-level",
    "Trigger and pre-selection" : "Passing preselection",
    "Dijet kinematic cuts" : "Passing dijet kinematics",
    "Jet substructure cuts" : "Passing jet substructure",
    "Jet $\\xi$ cuts" : "Passing xi(jets)",
    "2 multi-RP PPS protons" : "Passing 2 PPS multiRP",
    "Proton-jet matching (signal region A)" : "Passing WW (A)",
    "Proton-jet matching (signal region B)" : "Passing WW (B)"
}
name_and_cell_ZZ = {
    "Generator-level (all decay channels)" : "Generator-level",
    "Trigger and pre-selection" : "Passing preselection",
    "Dijet kinematic cuts" : "Passing dijet kinematics",
    "Jet substructure cuts" : "Passing jet substructure",
    "Jet $\\xi$ cuts" : "Passing xi(jets)",
    "2 multi-RP PPS protons" : "Passing 2 PPS multiRP",
    "Proton-jet matching (signal region A)" : "Passing ZZ (A)",
    "Proton-jet matching (signal region B)" : "Passing ZZ (B)"
}
name_and_cell_BG = {
    "Generator-level (all decay channels)" : "Generator-level",
    "Trigger and pre-selection" : "Passing preselection",
    "Dijet kinematic cuts" : "Passing dijet kinematics",
    "Jet substructure cuts" : "Passing jet substructure",
    "Jet $\\xi$ cuts" : "Passing xi(jets)",
    "WW cuts" : "Passing WW channel before PPS matching",
    "ZZ cuts" : "Passing ZZ channel before PPS matching",
}


In [27]:
results_and_year = {
    2016 : [i for i in results if i["Year"] == 2016],
    2017 : [i for i in results if i["Year"] == 2017],
    2018 : [i for i in results if i["Year"] == 2018]
}
bg_and_year = {
    2016 : [i for i in bgs if i["Year"] == 2016],
    2017 : [i for i in bgs if i["Year"] == 2017],
    2018 : [i for i in bgs if i["Year"] == 2018]
}
couplings_table = {
    "A0W" : ["WW_A0W5e-7","WW_A0W1e-6","WW_A0W2e-6","WW_A0W5e-6"],
    "ACW" : ["WW_ACW2e-6","WW_ACW5e-6","WW_ACW8e-6","WW_ACW2e-5"],
    "A0Z" : ["ZZ_A0Z1e-5","ZZ_A0Z2e-5","ZZ_A0Z5e-5"],
    "ACZ" : ["ZZ_ACZ1e-5","ZZ_ACZ2e-5","ZZ_ACZ5e-5"],
}
bgs_table = {
    #"BG" : ["QCD","others"],
    "BG" : ["all BGs"],
}

In [28]:
def my_notation(x):
    if x > 10:
        return "%.1f" % x
    elif x > 1:
        return "%.2f" % x
    else:
        return "%.2g" % x
    
for year in results_and_year:
    for coupling in couplings_table:
        print("Content for table " + coupling + " " + str(year))
        name_and_cell = name_and_cell_WW if coupling[-1] == "W" else name_and_cell_ZZ
        for label in name_and_cell: 
            row = ""
            row += label + " &"
            for aqgc in couplings_table[coupling]:
                result = [i for i in results_and_year[year] if i["AQGC point"] == aqgc]
                if len(result) != 0:
                    row += " "+my_notation(result[0][name_and_cell[label]])+" &" 
                else:
                    row += " not produced &"
            row = row[:-1] + "\\\\"
            print(row)
        print()
        
for bg in bgs_table["BG"]:
    print("Content for table " + bg)
    name_and_cell = name_and_cell_BG
    for label in name_and_cell: 
        row = ""
        row += label + " &"
        for year in bg_and_year:
            if bg == "all BGs":
                if name_and_cell[label] == "Generator-level":
                    result = bg_xsec_tot[str(year)] * 1000 * lumi[year]
                    row += " "+my_notation(result)+" &"
                else:    
                    result = sum([i[name_and_cell[label]] for i in bg_and_year[year]])
                    row += " "+my_notation(result)+" &"
            elif bg == "QCD":
                result = sum([i[name_and_cell[label]] for i in bg_and_year[year] if i["AQGC point"].startswith("qcd")])
                row += " "+my_notation(result)+" &"         
            elif bg == "others":
                result = sum([i[name_and_cell[label]] for i in bg_and_year[year] if not i["AQGC point"].startswith("qcd")])
                row += " "+my_notation(result)+" &"         
        row = row[:-1] + "\\\\"
        print(row)
    print()

Content for table A0W 2016
Generator-level (all decay channels) & 42.2 & 45.4 & 58.3 & 148.1 \\
Trigger and pre-selection & 0.36 & 1.40 & 5.67 & 35.5 \\
Dijet kinematic cuts & 0.27 & 1.04 & 4.16 & 26.0 \\
Jet substructure cuts & 0.12 & 0.45 & 1.85 & 11.4 \\
Jet $\xi$ cuts & 0.11 & 0.43 & 1.80 & 11.0 \\
2 multi-RP PPS protons & 0.0049 & 0.02 & 0.076 & 0.58 \\
Proton-jet matching (signal region A) & 0.0022 & 0.0066 & 0.023 & 0.17 \\
Proton-jet matching (signal region B) & 0.00048 & 0.0053 & 0.015 & 0.15 \\

Content for table ACW 2016
Generator-level (all decay channels) & 42.5 & 48.7 & 61.4 & 164.8 \\
Trigger and pre-selection & 0.45 & 2.69 & 6.94 & 42.9 \\
Dijet kinematic cuts & 0.33 & 2.00 & 5.13 & 31.6 \\
Jet substructure cuts & 0.14 & 0.9 & 2.28 & 14.0 \\
Jet $\xi$ cuts & 0.14 & 0.87 & 2.21 & 13.5 \\
2 multi-RP PPS protons & 0.0062 & 0.045 & 0.1 & 0.62 \\
Proton-jet matching (signal region A) & 0.0025 & 0.013 & 0.039 & 0.19 \\
Proton-jet matching (signal region B) & 0.00091 & 0.011 &